# Gaussian elimination - outer product formulation

This notebook shows how the [Gaussian elimination](https://en.wikipedia.org/wiki/Gaussian_elimination) iteratively transforms an unstructured linear system `Ax = y` into an equivalent triangular system `Bx = z` having the same solution `x`.

Let's consider a linear system `Ax = y` given by:

In [1]:
import numpy as np

In [2]:
A = np.array([[2.,1.,-1.],
              [-3.,-1.,2.],
              [-2.,1.,2.]])

In [3]:
y = np.array([8., -11., -3.])

The solution of this system is given by:

In [4]:
x = np.linalg.solve(A,y)

In [5]:
print(x)

[ 2.  3. -1.]


The first step in transforming the system `Ax = y` into an equivalent triangular system `Bx = z` is calculating an intermediate linear system:

$$
\underbrace{\left[ 
\begin{array}{ccc}
\times & \times & \times \\
0 & \times & \times \\
0 & \times & \times
\end{array} \right]}_{\mathbf{A}^{(1)}} \,
\underbrace{\left[ \begin{array}{c}
x_{1} \\
x_{2} \\
x_{3}
\end{array} \right]}_{\mathbf{x}} =
\underbrace{\left[
\begin{array}{c}
\times \\
\times \\
\times
\end{array} \right]
}_{\mathbf{y}^{(1)}} \: ,
$$

where the symbols "$\times$" denote non-null elements. Notice that the original matrix $\mathbf{A}$ is transformed into a matrix $\mathbf{A}^{(1)}$ having null-elements below the main diagonal in its first column. This zeroing procedure can be described as follows:

1. Calculate the factor $t^{(1)}_{1} = \frac{a_{10}}{a_{00}}$
2. Multiply the first row of $\mathbf{A}$ by the factor $t^{(1)}_{1}$ and subtract the result from the second row of $\mathbf{A}$
3. Multiply the first element of $\mathbf{y}$ by the factor $t^{(1)}_{1}$ and subtract the result from the second element of $\mathbf{y}$
4. Calculate the factor $t^{(1)}_{2} = \frac{a_{20}}{a_{00}}$
5. Multiply the first row of $\mathbf{A}$ by the factor $t^{(1)}_{2}$ and subtract the result from the third row of $\mathbf{A}$
6. Multiply the first element of $\mathbf{y}$ by the factor $t^{(1)}_{2}$ and subtract the result from the third element of $\mathbf{y}$

**P.S.:** the variable $a_{ij}$ represents the element $ij$ of the matrix $\mathbf{A}$

In [6]:
A1 = np.copy(A)
y1 = np.copy(y)

t11 = A[1,0]/A[0,0]    # step 1
A1[1,:] -= t11*A[0,:]  # step 2
y1[1] -= t11*y[0]      # step 3

t12 = A[2,0]/A[0,0]    # step 4
A1[2,:] -= t12*A[0,:]  # step 5
y1[2] -= t12*y[0]      # step 6

In [7]:
print(A1)

[[ 2.   1.  -1. ]
 [ 0.   0.5  0.5]
 [ 0.   2.   1. ]]


In [8]:
print(y1)

[8. 1. 5.]


The next step consists in calculating a linear system given by

$$
\underbrace{\left[ 
\begin{array}{ccc}
\times & \times & \times \\
0 & \times & \times \\
0 & 0 & \times
\end{array} \right]}_{\mathbf{A}^{(2)}} \,
\underbrace{\left[ \begin{array}{c}
x_{1} \\
x_{2} \\
x_{3}
\end{array} \right]}_{\mathbf{x}} =
\underbrace{\left[
\begin{array}{c}
\times \\
\times \\
\times
\end{array} \right]
}_{\mathbf{y}^{(2)}} \: .
$$

This zeroing procedure can be described is given by:

1. Calculate the factor $t^{(2)}_{2} = \frac{a_{21}^{(1)}}{a_{11}^{(1)}}$
2. Multiply the second row of $\mathbf{A}^{(1)}$ by the factor $t^{(2)}_{2}$ and subtract the result from the third row of $\mathbf{A}^{(1)}$
3. Multiply the second element of $\mathbf{y}^{(1)}$ by the factor $t^{(2)}_{2}$ and subtract the result from the third element of $\mathbf{y}^{(1)}$

**P.S.:** the variable $a_{ij}^{(1)}$ represents the element $ij$ of the matrix $\mathbf{A}^{(1)}$

In [9]:
A2 = np.copy(A1)
y2 = np.copy(y1)

t22 = A1[2,1]/A1[1,1]  # step 1
A2[2,:] -= t22*A1[1,:]  # step 2
y2[2] -= t22*y1[1]      # step 3

In [10]:
print(A2)

[[ 2.   1.  -1. ]
 [ 0.   0.5  0.5]
 [ 0.   0.  -1. ]]


In [11]:
print(y2)

[8. 1. 1.]


In this case, $\mathbf{B} = \mathbf{A}^{(2)}$ and $\mathbf{z} = \mathbf{y}^{(2)}$ form our desired equivalent system. The solution of this system is given by:

In [12]:
xeq = np.linalg.solve(A2,y2)

Compare this solution with the solution `x` of the original system

In [13]:
print(xeq)
print('\n')
print(x)

[ 2.  3. -1.]


[ 2.  3. -1.]


Notice that the proccess of calculating $\mathbf{A}^{(1)}$, $\mathbf{y}^{(1)}$, $\mathbf{A}^{(2)}$ and $\mathbf{y}^{(2)}$ can be represented by an iterative outer product update.

Specifically, our equivalent triangular system was iteratively calculated according to the following algorithm:

$$
\begin{array}{ccccc}
\mathbf{A}^{(0)} = \mathbf{A} & & & \mathbf{y}^{(0)} = \mathbf{y} \\\\
\mathbf{A}^{(1)} = \left(\mathbf{I} - \mathbf{M}^{(1)}\right) \mathbf{A}^{(0)} & & &
\mathbf{y}^{(1)} = \left(\mathbf{I} - \mathbf{M}^{(1)}\right) \mathbf{y}^{(0)} \\\\
\mathbf{A}^{(2)} = \left(\mathbf{I} - \mathbf{M}^{(2)}\right) \mathbf{A}^{(1)} & & &
\mathbf{y}^{(2)} = \left(\mathbf{I} - \mathbf{M}^{(2)}\right) \mathbf{y}^{(1)}
\end{array} \: ,$$

where $\mathbf{B} = \mathbf{A}^{(2)}$, $\mathbf{z} = \mathbf{y}^{(2)}$, $\mathbf{I}$ is the identity matrix of order 3, $\mathbf{M}^{(k)}$ is a matrix given by

$$\mathbf{M}^{(k)} = \mathbf{t}^{(k)} \cdot \left( \mathbf{u}^{(k-1)} \right)^{\top} \: ,$$

$\mathbf{u}^{(k-1)}$ is a $3 \times 1$ vector with all elements equal to $0$, except the element $k-1$, which is equal to $1$, and $\mathbf{t}^{(k)}$ is a $3 \times 1$ vector whose $i$th element $t_{i}^{(k)}$ is given by:

$t_{i}^{(k)} = \begin{cases} 0 & \quad \text{if } i < k \\\\ \dfrac{a^{(k-1)}_{ik}}{a^{(k-1)}_{kk}} & \quad \text{if } i \ge k\\ \end{cases} \: ,$

where $a^{(k-1)}_{ij}$ is the $ij$ element of the matrix $\mathbf{A}^{(k-1)}$. In matrix form, the non-null elements of $\mathbf{t}^{(k)}$ are given by:

$$
\mathbf{t}^{(k)}[k:] = \frac{1}{\mathbf{A}[k-1, k-1]} \mathbf{A}^{(k-1)}[k:] \quad .
$$

This proccess of transforming the original system into an equivalent triangular system requires that all the elements $a_{kk}^{(k-1)}$ be nonzero. They are called **pivots**.

Notice that the number of iterations needed for transforming the original system $\mathbf{A}\mathbf{x} = \mathbf{y}$ into the equivalent triangular system $\mathbf{B}\mathbf{x} = \mathbf{z}$ is equal to $N - 1$, where $N$ is the order of the square matrix $\mathbf{A}$.

The matrix $\left(\mathbf{I} - \mathbf{M}^{(k)}\right)$ is called **Gauss transformation**, the vector $\mathbf{t}^{(k)}$ is called **Gauss vector** and the non-null elements of $\mathbf{t}^{(k)}$ are called **Gauss multipliers** (Golub and Van Loan, 2013).

The code below solves our linear system $\mathbf{A}\mathbf{x} = \mathbf{y}$ by applying the Gaussian elimination formulated as an outer product:

In [14]:
I = np.identity(3)

**Iteration k = 1:**

In [15]:
u0 = np.array([1., 0., 0.])

In [16]:
t1 = np.array([0., A[1][0]/A[0][0], A[2][0]/A[0][0]])

In [17]:
A1 = (I - np.outer(t1, u0)).dot(A)

In [18]:
print(t1)

[ 0.  -1.5 -1. ]


In [19]:
print(A1)

[[ 2.   1.  -1. ]
 [ 0.   0.5  0.5]
 [ 0.   2.   1. ]]


In [20]:
y1 = (I - np.outer(t1, u0)).dot(y)

In [21]:
print(y1)

[8. 1. 5.]


**Iteration k = 2:**

In [22]:
u1 = np.array([0., 1., 0.])

In [23]:
t2 = np.array([0., 0., A1[2][1]/A1[1][1]])

In [24]:
print(t2)

[0. 0. 4.]


In [25]:
B = (I - np.outer(t2, u1)).dot(A1)

In [26]:
print(B)

[[ 2.   1.  -1. ]
 [ 0.   0.5  0.5]
 [ 0.   0.  -1. ]]


In [27]:
z = (I - np.outer(t2, u1)).dot(y1)

In [28]:
print(z)

[8. 1. 1.]


Solution of this equivalent triangular system:

In [29]:
print(np.linalg.solve(B,z))

[ 2.  3. -1.]


Solution of the original system:

In [30]:
print(np.linalg.solve(A,y))

[ 2.  3. -1.]
